### Data Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# 1. Load electricity
electricity = pd.read_csv('/content/drive/MyDrive/BGD2_Project/electricity_cleaned.csv')

# 2. Find building columns (exclude timestamp)
building_columns = [col for col in electricity.columns if col != 'timestamp']

# 3. Define building type keywords you care about
keywords = ['office', 'education', 'lodging', 'health', 'retail', 'assembly']

# 4. Select columns that match any of the keywords
selected_buildings = [col for col in building_columns if any(kw in col.lower() for kw in keywords)]

# 5. Keep timestamp + selected buildings
columns_to_keep = ['timestamp'] + selected_buildings
electricity_filtered = electricity[columns_to_keep]

# 6. NOW melt into long format (timestamp, building_id, electricity_kwh)
electricity_long = electricity_filtered.melt(
    id_vars='timestamp',
    var_name='building_id',
    value_name='electricity_kwh'
)

# 7. Save this long format file
output_path = '/content/drive/MyDrive/BGD2_Project/electricity_filtered_melted.csv'
electricity_long.to_csv(output_path, index=False)

# 8. Confirm how many buildings and preview
print(f"Number of buildings selected: {len(selected_buildings)}")
electricity_long.head()



Number of buildings selected: 1290


,timestamp,building_id,electricity_kwh
0,2016-01-01 00:00:00,Panther_lodging_Cora,NaN
1,2016-01-01 01:00:00,Panther_lodging_Cora,NaN
2,2016-01-01 02:00:00,Panther_lodging_Cora,NaN
3,2016-01-01 03:00:00,Panther_lodging_Cora,NaN
4,2016-01-01 04:00:00,Panther_lodging_Cora,NaN


In [ ]:
import pandas as pd

# 1. Load the melted electricity file
df = pd.read_csv('/content/drive/MyDrive/BGD2_Project/electricity_filtered_melted.csv')

# 2. Define keywords and their corresponding types
type_map = {
    'lodging': 'Lodging',
    'education': 'Education',
    'office': 'Office',
    'health': 'Health',
    'retail': 'Retail',
    'assembly': 'Assembly'
}

# 3. Create building_type column by mapping keywords in building_id
def infer_building_type(name):
    for keyword, type_name in type_map.items():
        if keyword in name.lower():
            return type_name
    return 'Other'

df['building_type'] = df['building_id'].apply(infer_building_type)

# 4. Save the updated file
df.to_csv('/content/drive/MyDrive/BGD2_Project/electricity_with_type.csv', index=False)

# 5. Preview
df.head()


,timestamp,building_id,electricity_kwh,building_type
0,2016-01-01 00:00:00,Panther_lodging_Cora,NaN,Lodging
1,2016-01-01 01:00:00,Panther_lodging_Cora,NaN,Lodging
2,2016-01-01 02:00:00,Panther_lodging_Cora,NaN,Lodging
3,2016-01-01 03:00:00,Panther_lodging_Cora,NaN,Lodging
4,2016-01-01 04:00:00,Panther_lodging_Cora,NaN,Lodging


In [ ]:
import pandas as pd

# 1. Load electricity with type already assigned
electricity = pd.read_csv('/content/drive/MyDrive/BGD2_Project/electricity_with_type.csv')

# 2. Load and melt water_cleaned
water = pd.read_csv('/content/drive/MyDrive/BGD2_Project/water_cleaned.csv')

# 3. Melt water to long format
water_long = water.melt(
    id_vars='timestamp',
    var_name='building_id',
    value_name='water_units'  # We'll treat this as energy impact
)

# 4. Merge water into electricity file using timestamp + building_id
merged = pd.merge(electricity, water_long, on=['timestamp', 'building_id'], how='left')

# 5. Save merged version
merged.to_csv('/content/drive/MyDrive/BGD2_Project/electricity_with_water.csv', index=False)

# 6. Preview
merged.head()


,timestamp,building_id,electricity_kwh,building_type,water_units
0,2016-01-01 00:00:00,Panther_lodging_Cora,NaN,Lodging,378.541
1,2016-01-01 01:00:00,Panther_lodging_Cora,NaN,Lodging,378.541
2,2016-01-01 02:00:00,Panther_lodging_Cora,NaN,Lodging,757.082
3,2016-01-01 03:00:00,Panther_lodging_Cora,NaN,Lodging,757.082
4,2016-01-01 04:00:00,Panther_lodging_Cora,NaN,Lodging,757.082


In [ ]:
import pandas as pd

# 1. Load your current electricity + water merged file
merged = pd.read_csv('/content/drive/MyDrive/BGD2_Project/electricity_with_water.csv')

# 2. Load the gas meter data
gas = pd.read_csv('/content/drive/MyDrive/BGD2_Project/gas_cleaned.csv')

# 3. Melt the gas data to long format
gas_long = gas.melt(
    id_vars='timestamp',
    var_name='building_id',
    value_name='gas_units'  # New column to hold gas readings
)

# 4. Merge gas data into your merged electricity+water file
merged_with_gas = pd.merge(merged, gas_long, on=['timestamp', 'building_id'], how='left')

# 5. Save the new merged file
merged_with_gas.to_csv('/content/drive/MyDrive/BGD2_Project/electricity_water_gas_merged.csv', index=False)

# 6. Preview a few rows
merged_with_gas.head()

,timestamp,building_id,electricity_kwh,building_type,water_units,gas_units
0,2016-01-01 00:00:00,Panther_lodging_Cora,NaN,Lodging,378.541,NaN
1,2016-01-01 01:00:00,Panther_lodging_Cora,NaN,Lodging,378.541,NaN
2,2016-01-01 02:00:00,Panther_lodging_Cora,NaN,Lodging,757.082,NaN
3,2016-01-01 03:00:00,Panther_lodging_Cora,NaN,Lodging,757.082,NaN
4,2016-01-01 04:00:00,Panther_lodging_Cora,NaN,Lodging,757.082,NaN


In [ ]:
import pandas as pd

# 1. Load your energy file
energy_data = pd.read_csv('/content/drive/MyDrive/BGD2_Project/electricity_water_gas_merged.csv')  # <-- your electricity_water_gas_merged.csv

# 2. Load the weather file
weather_data = pd.read_csv('/content/drive/MyDrive/BGD2_Project/weather.csv')

# 3. Make sure timestamps are datetime
energy_data['timestamp'] = pd.to_datetime(energy_data['timestamp'])
weather_data['timestamp'] = pd.to_datetime(weather_data['timestamp'])

# 4. Extract site_id from building_id
energy_data['site_id'] = energy_data['building_id'].str.extract(r"^([A-Za-z]+)_")

# 5. Merge based on BOTH timestamp and site_id
merged_data = pd.merge(
    energy_data,
    weather_data,
    how='left',
    on=['timestamp', 'site_id']  # <-- this matches correctly
)

# 6. Save the result
merged_data.to_csv('/content/drive/MyDrive/BGD2_Project/final_energy_weather_merged.csv', index=False)

# 7. Quick preview
print(merged_data.head())

            timestamp           building_id  electricity_kwh building_type  \
0 2016-01-01 00:00:00  Panther_lodging_Cora              NaN       Lodging   
1 2016-01-01 01:00:00  Panther_lodging_Cora              NaN       Lodging   
2 2016-01-01 02:00:00  Panther_lodging_Cora              NaN       Lodging   
3 2016-01-01 03:00:00  Panther_lodging_Cora              NaN       Lodging   
4 2016-01-01 04:00:00  Panther_lodging_Cora              NaN       Lodging   

   water_units  gas_units  site_id  airTemperature  cloudCoverage  \
0      378.541        NaN  Panther            19.4            NaN   
1      378.541        NaN  Panther            21.1            6.0   
2      757.082        NaN  Panther            21.1            NaN   
3      757.082        NaN  Panther            20.6            NaN   
4      757.082        NaN  Panther            21.1            NaN   

   dewTemperature  precipDepth1HR  precipDepth6HR  seaLvlPressure  \
0            19.4             0.0             N

In [ ]:
import pandas as pd

# 1. Load your final energy-weather merged file
energy_weather = pd.read_csv('/content/drive/MyDrive/BGD2_Project/final_energy_weather_merged.csv')

# 2. Load the metadata file
metadata = pd.read_csv('/content/drive/MyDrive/BGD2_Project/metadata.csv')

# 3. Select only building_id and eui columns
metadata_eui = metadata[['building_id', 'eui']]

# 4. Merge by building_id
final_merged_with_eui = pd.merge(
    energy_weather,
    metadata_eui,
    on='building_id',
    how='left'  # Left join to keep all energy-weather rows
)

# 5. Save the new file
final_merged_with_eui.to_csv('/content/drive/MyDrive/BGD2_Project/final_energy_weather_with_eui.csv', index=False)

# 6. Preview
final_merged_with_eui.head()


### Data Cleaning & Merging Report

#### Objective

The purpose of the data cleaning process was to prepare electricity, water, gas, weather, and building characteristic data from the Building Data Genome 2 (BDG2) project for analysis. The goal was to restructure the data for easier merging, align it consistently, and enrich it with key building information (like Energy Use Intensity, EUI) for trend analysis, anomaly detection, and predictive modeling.

#### Identifying Missing Values, Anomalies, and Inconsistencies

- Missing Values:
  Meter datasets contained missing values (NaN) for some buildings and timestamps.
  Weather readings for `airTemperature` were assumed to have complete hourly coverage.
- Anomalies:
  Long sequences of zero readings were observed, particularly in water and gas meter files, possibly due to seasonal building inactivity or meter downtime.
- Inconsistencies:
  Different buildings had varying amounts of valid data.
  Building types were not labeled originally and needed to be inferred from building IDs.

#### How the Data Was Cleaned and Prepared

#### Meter Data (Electricity, Water, Gas)
- Electricity Data:
  - Loaded `electricity_cleaned.csv`.
  - Selected buildings containing keywords: `office`, `education`, `lodging`, `health`, `retail`, `assembly`.
  - Reshaped into long format: `timestamp`, `building_id`, `electricity_kwh`.
  - Inferred `building_type` from keywords in the building name.
- Water Data:
  - Loaded `water_cleaned.csv`.
  - Melted into long format: `timestamp`, `building_id`, `water_units`.
  - Merged water readings into electricity data using `timestamp` and `building_id`.
- Gas Data:
  - Loaded `gas_cleaned.csv`.
  - Melted into long format: `timestamp`, `building_id`, `gas_units`.
  - Merged gas readings into the combined electricity+water data using `timestamp` and `building_id`.

#### Weather Data (Air Temperature)
- Loaded `weather.csv`.
- Weather data was merged by `timestamp` only.
- The `airTemperature` was duplicated across all buildings at each timestamps.

#### Building Characteristic Data (EUI)
- Loaded `metadata.csv`
- Selected `building_id` and `eui` columns
- merged `eui` values in to the final dataset by matching `building_id`.
- EUI (Energy Use Intensity, measured in kWh/year/m^2) was added as a new feature for each building record.
- Some buildings had missing `eui` values.

#### Final Merged Dataset
- Final dataset columns:
  - `timestamp`
  - `building_id`
  - `electricity_kwh`
  - `water_units`
  - `gas_units`
  - `airTemperature`
  - Missing values were preserved without artificial filling.
  - No interpolation or manual adjustment was performed.